In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1206035,2021-04-22,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1206036,2021-04-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1206037,2021-04-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1206038,2021-04-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
32259,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32261,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32263,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32265,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
32267,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
742617,2021-04-22,Arkansas,Arkansas,5001,2089,36.00,5000,Arkansas,AR,Arkansas,State,3017804
742619,2021-04-23,Arkansas,Arkansas,5001,2089,36.00,5000,Arkansas,AR,Arkansas,State,3017804
742621,2021-04-24,Arkansas,Arkansas,5001,2092,36.00,5000,Arkansas,AR,Arkansas,State,3017804
742623,2021-04-25,Arkansas,Arkansas,5001,2092,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1206035,2021-04-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1206036,2021-04-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1206037,2021-04-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1206038,2021-04-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-26') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
461,2021-04-26,Snohomish,Washington,35313,564.00,53061,WA,County,822083,4295.55,68.61
920,2021-04-26,Cook,Illinois,530003,10455.00,17031,IL,County,5150233,10290.85,203.00
1378,2021-04-26,Orange,California,269724,4933.00,6059,CA,County,3175692,8493.39,155.34
1835,2021-04-26,Maricopa,Arizona,534982,9836.00,4013,AZ,County,4485414,11927.15,219.29
2292,2021-04-26,Los Angeles,California,1232469,23799.00,6037,CA,County,10039107,12276.68,237.06
...,...,...,...,...,...,...,...,...,...,...,...
1205379,2021-04-26,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08
1205575,2021-04-26,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1205740,2021-04-26,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1205901,2021-04-26,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
461,2021-04-26,Snohomish,Washington,35313,564.00,53061,WA,County,822083,4295.55,68.61,68.61,4295.55
920,2021-04-26,Cook,Illinois,530003,10455.00,17031,IL,County,5150233,10290.85,203.00,203.00,10290.85
1378,2021-04-26,Orange,California,269724,4933.00,6059,CA,County,3175692,8493.39,155.34,155.34,8493.39
1835,2021-04-26,Maricopa,Arizona,534982,9836.00,4013,AZ,County,4485414,11927.15,219.29,219.29,11927.15
2292,2021-04-26,Los Angeles,California,1232469,23799.00,6037,CA,County,10039107,12276.68,237.06,237.06,12276.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205379,2021-04-26,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08,75.08,1951.95
1205575,2021-04-26,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1205740,2021-04-26,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1205901,2021-04-26,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
776506,2021-04-26,Hale,Alabama,2201,75.00,1065,AL,County,14651,15022.87,511.91,511.91,15022.87,1
845201,2021-04-26,Clarke,Alabama,3499,60.00,1025,AL,County,23622,14812.46,254.00,254.00,14812.46,2
607581,2021-04-26,Lowndes,Alabama,1383,53.00,1085,AL,County,9726,14219.62,544.93,544.93,14219.62,3
452076,2021-04-26,Franklin,Alabama,4221,82.00,1059,AL,County,31362,13458.96,261.46,261.46,13458.96,4
538733,2021-04-26,Etowah,Alabama,13737,352.00,1055,AL,County,102268,13432.35,344.19,344.19,13432.35,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143528,2021-04-26,Platte,Wyoming,620,11.00,56031,WY,County,8393,7387.11,131.06,131.06,7387.11,19
741048,2021-04-26,Converse,Wyoming,990,17.00,56009,WY,County,13822,7162.49,122.99,122.99,7162.49,20
921253,2021-04-26,Lincoln,Wyoming,1391,12.00,56023,WY,County,19830,7014.62,60.51,60.51,7014.62,21
1025289,2021-04-26,Niobrara,Wyoming,151,2.00,56027,WY,County,2356,6409.17,84.89,84.89,6409.17,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
607581,2021-04-26,Lowndes,Alabama,1383,53.00,1085,AL,County,9726,14219.62,544.93,544.93,14219.62,3,1
776506,2021-04-26,Hale,Alabama,2201,75.00,1065,AL,County,14651,15022.87,511.91,511.91,15022.87,1,2
258434,2021-04-26,Walker,Alabama,7176,278.00,1127,AL,County,63521,11297.05,437.65,437.65,11297.05,28,3
537141,2021-04-26,Clay,Alabama,1521,56.00,1027,AL,County,13235,11492.26,423.12,423.12,11492.26,23,4
655218,2021-04-26,Greene,Alabama,920,34.00,1063,AL,County,8111,11342.62,419.18,419.18,11342.62,26,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653633,2021-04-26,Hot Springs,Wyoming,366,3.00,56017,WY,County,4413,8293.68,67.98,67.98,8293.68,17,19
921253,2021-04-26,Lincoln,Wyoming,1391,12.00,56023,WY,County,19830,7014.62,60.51,60.51,7014.62,21,20
898302,2021-04-26,Uinta,Wyoming,2165,12.00,56041,WY,County,20226,10704.04,59.33,59.33,10704.04,5,21
257222,2021-04-26,Teton,Wyoming,3724,9.00,56039,WY,County,23464,15871.12,38.36,38.36,15871.12,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4295.55,19,23
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4295.55,19,23
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4295.55,19,23
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4295.55,19,23
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4295.55,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203207,2021-04-22,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1203208,2021-04-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1203209,2021-04-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1203210,2021-04-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
773286,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15022.87,2,1,1.00
773287,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15022.87,2,1,0.00
773288,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15022.87,2,1,0.00
773289,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,15022.87,2,1,0.00
773290,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,15022.87,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936657,2021-04-22,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5656.65,6,23,0.00
936658,2021-04-23,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5656.65,6,23,0.00
936659,2021-04-24,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5656.65,6,23,0.00
936660,2021-04-25,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5656.65,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
605147,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,1.00,0.00
605148,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
605149,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
605150,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
605151,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14219.62,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603157,2021-04-22,Albany,Wyoming,4170,11.00,56001,WY,County,38880,10725.31,28.29,28.29,10797.33,23,4,15.00,0.00
603158,2021-04-23,Albany,Wyoming,4184,11.00,56001,WY,County,38880,10761.32,28.29,28.29,10797.33,23,4,14.00,0.00
603159,2021-04-24,Albany,Wyoming,4184,11.00,56001,WY,County,38880,10761.32,28.29,28.29,10797.33,23,4,0.00,0.00
603160,2021-04-25,Albany,Wyoming,4184,11.00,56001,WY,County,38880,10761.32,28.29,28.29,10797.33,23,4,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-26') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
308498,2021-04-26,Imperial,California,28136,719.00,6025,CA,County,181215,15526.31,396.77,396.77,15526.31,1,2,12.00,0.00
2292,2021-04-26,Los Angeles,California,1232469,23799.00,6037,CA,County,10039107,12276.68,237.06,237.06,12276.68,2,5,274.00,-3.00
153628,2021-04-26,San Bernardino,California,295565,4571.00,6071,CA,County,2180085,13557.50,209.67,209.67,13557.50,3,4,55.00,3.00
540277,2021-04-26,Inyo,California,1396,36.00,6027,CA,County,18039,7738.79,199.57,199.57,7738.79,4,26,0.00,0.00
68705,2021-04-26,Stanislaus,California,61025,1039.00,6099,CA,County,550660,11082.16,188.68,188.68,11082.16,5,9,272.00,3.00
31842,2021-04-26,Riverside,California,298001,4559.00,6065,CA,County,2470546,12062.15,184.53,184.53,12062.15,6,6,120.00,0.00
85182,2021-04-26,Tulare,California,49515,833.00,6107,CA,County,466195,10621.09,178.68,178.68,10621.09,7,10,22.00,1.00
54665,2021-04-26,San Joaquin,California,71987,1346.00,6077,CA,County,762148,9445.28,176.61,176.61,9445.28,8,19,15.00,6.00
31010,2021-04-26,Fresno,California,101061,1662.00,6019,CA,County,999101,10115.19,166.35,166.35,10115.19,9,14,70.00,0.00
489621,2021-04-26,Merced,California,31452,454.00,6047,CA,County,277680,11326.71,163.50,163.50,11326.71,10,8,52.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1143079,2021-04-26,Lassen,California,5685,1.00,6035,CA,County,30573,18594.84,3.27,3.27,18594.84,56,1,0.00,0.00
308498,2021-04-26,Imperial,California,28136,719.00,6025,CA,County,181215,15526.31,396.77,396.77,15526.31,1,2,12.00,0.00
655160,2021-04-26,Kings,California,22847,246.00,6031,CA,County,152940,14938.54,160.85,160.85,14938.54,11,3,6.00,0.00
153628,2021-04-26,San Bernardino,California,295565,4571.00,6071,CA,County,2180085,13557.50,209.67,209.67,13557.50,3,4,55.00,3.00
2292,2021-04-26,Los Angeles,California,1232469,23799.00,6037,CA,County,10039107,12276.68,237.06,237.06,12276.68,2,5,274.00,-3.00
31842,2021-04-26,Riverside,California,298001,4559.00,6065,CA,County,2470546,12062.15,184.53,184.53,12062.15,6,6,120.00,0.00
193149,2021-04-26,Kern,California,108313,1345.00,6029,CA,County,900202,12032.08,149.41,149.41,12032.08,14,7,33.00,0.00
489621,2021-04-26,Merced,California,31452,454.00,6047,CA,County,277680,11326.71,163.50,163.50,11326.71,10,8,52.00,0.00
68705,2021-04-26,Stanislaus,California,61025,1039.00,6099,CA,County,550660,11082.16,188.68,188.68,11082.16,5,9,272.00,3.00
85182,2021-04-26,Tulare,California,49515,833.00,6107,CA,County,466195,10621.09,178.68,178.68,10621.09,7,10,22.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
308498,2021-04-26,Imperial,California,28136,719.00,6025,CA,County,181215,15526.31,396.77,396.77,15526.31,1,2,12.00,0.00
2292,2021-04-26,Los Angeles,California,1232469,23799.00,6037,CA,County,10039107,12276.68,237.06,237.06,12276.68,2,5,274.00,-3.00
153628,2021-04-26,San Bernardino,California,295565,4571.00,6071,CA,County,2180085,13557.50,209.67,209.67,13557.50,3,4,55.00,3.00
540277,2021-04-26,Inyo,California,1396,36.00,6027,CA,County,18039,7738.79,199.57,199.57,7738.79,4,26,0.00,0.00
68705,2021-04-26,Stanislaus,California,61025,1039.00,6099,CA,County,550660,11082.16,188.68,188.68,11082.16,5,9,272.00,3.00
31842,2021-04-26,Riverside,California,298001,4559.00,6065,CA,County,2470546,12062.15,184.53,184.53,12062.15,6,6,120.00,0.00
85182,2021-04-26,Tulare,California,49515,833.00,6107,CA,County,466195,10621.09,178.68,178.68,10621.09,7,10,22.00,1.00
54665,2021-04-26,San Joaquin,California,71987,1346.00,6077,CA,County,762148,9445.28,176.61,176.61,9445.28,8,19,15.00,6.00
31010,2021-04-26,Fresno,California,101061,1662.00,6019,CA,County,999101,10115.19,166.35,166.35,10115.19,9,14,70.00,0.00
489621,2021-04-26,Merced,California,31452,454.00,6047,CA,County,277680,11326.71,163.50,163.50,11326.71,10,8,52.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.77,15526.31,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.77,15526.31,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.77,15526.31,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.77,15526.31,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.77,15526.31,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4042,04/22/21,Lassen,5679,1.00,18575.21,3.27,3.27,18594.84,56,1,0.00,0.00
4043,04/23/21,Lassen,5685,1.00,18594.84,3.27,3.27,18594.84,56,1,6.00,0.00
4044,04/24/21,Lassen,5685,1.00,18594.84,3.27,3.27,18594.84,56,1,0.00,0.00
4045,04/25/21,Lassen,5685,1.00,18594.84,3.27,3.27,18594.84,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.77,15526.31,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.77,15526.31,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.77,15526.31,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.77,15526.31,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.77,15526.31,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4128,04/22/21,Merced,31358,454.00,11292.86,163.50,163.50,11326.71,10,8,37.00,0.00
4129,04/23/21,Merced,31400,454.00,11307.98,163.50,163.50,11326.71,10,8,42.00,0.00
4130,04/24/21,Merced,31400,454.00,11307.98,163.50,163.50,11326.71,10,8,0.00,0.00
4131,04/25/21,Merced,31400,454.00,11307.98,163.50,163.50,11326.71,10,8,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)